# First activity

#### Libraries importing

In [148]:
import pandas as pd # library for data analysis
import requests # library to handle requests
import numpy as np
import matplotlib.cm as cm
import matplotlib.colors as colors
import folium
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim
from bs4 import BeautifulSoup # library to parse HTML documents

print("Packages imported")

Packages imported


#### Get the response in the form of html

In [149]:
url="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
table_class="wikitable sortable jquery-tablesorter"
response=requests.get(url)
print(response.status_code) #Check if the code is 200

200


#### Parse data from the html into a beautifulsoup object

In [150]:
soup = BeautifulSoup(response.text, 'html.parser')
indiatable=soup.find('table',{'class':"wikitable"})
print("Ok")

Ok


#### Conversion to dataframe

In [151]:
df=pd.read_html(str(indiatable))
# convert list to dataframe
df=pd.DataFrame(df[0])
df.head()

Postal Code           Borough              Neighbourhood
0         M1A      Not assigned               Not assigned
1         M2A      Not assigned               Not assigned
2         M3A        North York                  Parkwoods
3         M4A        North York           Victoria Village
4         M5A  Downtown Toronto  Regent Park, Harbourfront

#### Data wrangling

In [152]:
df = df[df["Borough"]!="Not assigned"] # Get rid of not assigned boroughs.
df = df.reset_index(drop=True)
df.head(12)

Postal Code           Borough  \
0          M3A        North York   
1          M4A        North York   
2          M5A  Downtown Toronto   
3          M6A        North York   
4          M7A  Downtown Toronto   
5          M9A         Etobicoke   
6          M1B       Scarborough   
7          M3B        North York   
8          M4B         East York   
9          M5B  Downtown Toronto   
10         M6B        North York   
11         M9B         Etobicoke   

                                        Neighbourhood  
0                                           Parkwoods  
1                                    Victoria Village  
2                           Regent Park, Harbourfront  
3                    Lawrence Manor, Lawrence Heights  
4         Queen's Park, Ontario Provincial Government  
5             Islington Avenue, Humber Valley Village  
6                                      Malvern, Rouge  
7                                           Don Mills  
8                     Parkview Hill, Woodbine Gardens  
9                            Garden District, Ryerson  
10                                          Glencairn  
11  West Deane Park, Princess Gardens, Martin Grov...

In [153]:
df[df["Neighbourhood"]=="Not assigned"] # Check if there are not assigned neighbourhoods.

Empty DataFrame
Columns: [Postal Code, Borough, Neighbourhood]
Index: []

In [154]:
print("Number of rows:",df.shape[0])

Number of rows: 103


# Second activity

#### Import data from csv

In [155]:
url = "https://cocl.us/Geospatial_data"
coord = pd.read_csv(url) # Import coordinates from csv file. Could not get the coordinates with Geocoder.
coord.head()

Postal Code   Latitude  Longitude
0         M1B  43.806686 -79.194353
1         M1C  43.784535 -79.160497
2         M1E  43.763573 -79.188711
3         M1G  43.770992 -79.216917
4         M1H  43.773136 -79.239476

#### Data frames merging

In [156]:
df = df.merge(coord,on = "Postal Code") # Merge the dataframes using Postal Code as the column to join on.

In [157]:
df.head()

Postal Code           Borough                                Neighbourhood  \
0         M3A        North York                                    Parkwoods   
1         M4A        North York                             Victoria Village   
2         M5A  Downtown Toronto                    Regent Park, Harbourfront   
3         M6A        North York             Lawrence Manor, Lawrence Heights   
4         M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government   

    Latitude  Longitude  
0  43.753259 -79.329656  
1  43.725882 -79.315572  
2  43.654260 -79.360636  
3  43.718518 -79.464763  
4  43.662301 -79.389494

# Third activity

#### Only boroughs which contain "Toronto" in their names

In [158]:
df_toronto = df[df["Borough"].str.contains("Toronto")]
df_toronto = df_toronto.reset_index(drop=True)
df_toronto.head()

Postal Code           Borough                                Neighbourhood  \
0         M5A  Downtown Toronto                    Regent Park, Harbourfront   
1         M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government   
2         M5B  Downtown Toronto                     Garden District, Ryerson   
3         M5C  Downtown Toronto                               St. James Town   
4         M4E      East Toronto                                  The Beaches   

    Latitude  Longitude  
0  43.654260 -79.360636  
1  43.662301 -79.389494  
2  43.657162 -79.378937  
3  43.651494 -79.375418  
4  43.676357 -79.293031

#### Get geographical coordinates of Toronto

In [176]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Toronto are 43.6534817, -79.3839347.


#### Toronto neighbourhoods visualizing

In [177]:
# create map of Toronto using latitude and longitude values

map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Borough'], df_toronto['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

#### Foursquare credentials and version

In [162]:
CLIENT_ID = 'UWQ442YGO0FX5C0YNVUJ5D4MPOTG1OH3JYYIKJX5MTO0H5WF' # your Foursquare ID
CLIENT_SECRET = '2VQ4USSTMSXCN3KUTAQI1CGF3SCM230LFIAQNMPSMCFYT13F' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Credentails:
CLIENT_ID: UWQ442YGO0FX5C0YNVUJ5D4MPOTG1OH3JYYIKJX5MTO0H5WF
CLIENT_SECRET:2VQ4USSTMSXCN3KUTAQI1CGF3SCM230LFIAQNMPSMCFYT13F


#### Function to get top 100 venues within 500m for all neighbourhoods

In [163]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Create dataframe toronto_venues which contains the results of the function defined above

In [164]:
toronto_venues = getNearbyVenues(names=df_toronto['Neighbourhood'],
                                   latitudes=df_toronto['Latitude'],
                                   longitudes=df_toronto['Longitude']
                                  )

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West, Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
R

#### Venues returned for each neighbourhood

In [165]:
toronto_venues.groupby('Neighborhood').count()

Neighborhood Latitude  \
Neighborhood                                                                
Berczy Park                                                            55   
Brockton, Parkdale Village, Exhibition Place                           23   
Business reply mail Processing Centre, South Ce...                     16   
CN Tower, King and Spadina, Railway Lands, Harb...                     16   
Central Bay Street                                                     68   
Christie                                                               16   
Church and Wellesley                                                   75   
Commerce Court, Victoria Hotel                                        100   
Davisville                                                             33   
Davisville North                                                        9   
Dufferin, Dovercourt Village                                           13   
First Canadian Place, Underground city                                100   
Forest Hill North & West, Forest Hill Road Park                         4   
Garden District, Ryerson                                              100   
Harbourfront East, Union Station, Toronto Islands                     100   
High Park, The Junction South                                          25   
India Bazaar, The Beaches West                                         19   
Kensington Market, Chinatown, Grange Park                              74   
Lawrence Park                                                           3   
Little Portugal, Trinity                                               45   
Moore Park, Summerhill East                                             2   
North Toronto West, Lawrence Park                                      18   
Parkdale, Roncesvalles                                                 14   
Queen's Park, Ontario Provincial Government                            33   
Regent Park, Harbourfront                                              44   
Richmond, Adelaide, King                                              100   
Rosedale                                                                4   
Roselawn                                                                2   
Runnymede, Swansea                                                     33   
St. James Town                                                         85   
St. James Town, Cabbagetown                                            48   
Stn A PO Boxes                                                         96   
Studio District                                                        37   
Summerhill West, Rathnelly, South Hill, Forest ...                     14   
The Annex, North Midtown, Yorkville                                    19   
The Beaches                                                             4   
The Danforth West, Riverdale                                           43   
Toronto Dominion Centre, Design Exchange                              100   
University of Toronto, Harbord                                         34   

                                                    Neighborhood Longitude  \
Neighborhood                                                                 
Berczy Park                                                             55   
Brockton, Parkdale Village, Exhibition Place                            23   
Business reply mail Processing Centre, South Ce...                      16   
CN Tower, King and Spadina, Railway Lands, Harb...                      16   
Central Bay Street                                                      68   
Christie                                                                16   
Church and Wellesley                                                    75   
Commerce Court, Victoria Hotel                                         100   
Davisville                                                              33   
Davisville North                    

#### Number of unique categories

In [166]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 235 uniques categories.


#### Analysis of each neighborhood

In [167]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

Yoga Studio  Afghan Restaurant  Airport  Airport Food Court  Airport Gate  \
0            0                  0        0                   0             0   
1            0                  0        0                   0             0   
2            0                  0        0                   0             0   
3            0                  0        0                   0             0   
4            0                  0        0                   0             0   

   Airport Lounge  Airport Service  Airport Terminal  American Restaurant  \
0               0                0                 0                    0   
1               0                0                 0                    0   
2               0                0                 0                    0   
3               0                0                 0                    0   
4               0                0                 0                    0   

   Antique Shop  ...  Thai Restaurant  Theater  Theme Restaurant  \
0             0  ...                0        0                 0   
1             0  ...                0        0                 0   
2             0  ...                0        0                 0   
3             0  ...                0        0                 0   
4             0  ...                0        0                 0   

   Toy / Game Store  Trail  Train Station  Vegetarian / Vegan Restaurant  \
0                 0      0              0                              0   
1                 0      0              0                              0   
2                 0      0              0                              0   
3                 0      0              0                              0   
4                 0      0              0                              0   

   Video Game Store  Vietnamese Restaurant  Wine Bar  
0                 0                      0         0  
1                 0                      0         0  
2                 0                      0         0  
3                 0                      0         0  
4                 0                      0         0  

[5 rows x 235 columns]

#### Group rows by neighborhood and take the mean of the frequency of occurrence of each category

In [168]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

Neighborhood  Yoga Studio  \
0                                         Berczy Park     0.000000   
1        Brockton, Parkdale Village, Exhibition Place     0.000000   
2   Business reply mail Processing Centre, South C...     0.000000   
3   CN Tower, King and Spadina, Railway Lands, Har...     0.000000   
4                                  Central Bay Street     0.014706   
5                                            Christie     0.000000   
6                                Church and Wellesley     0.026667   
7                      Commerce Court, Victoria Hotel     0.000000   
8                                          Davisville     0.000000   
9                                    Davisville North     0.000000   
10                       Dufferin, Dovercourt Village     0.000000   
11             First Canadian Place, Underground city     0.000000   
12    Forest Hill North & West, Forest Hill Road Park     0.000000   
13                           Garden District, Ryerson     0.000000   
14  Harbourfront East, Union Station, Toronto Islands     0.000000   
15                      High Park, The Junction South     0.000000   
16                     India Bazaar, The Beaches West     0.000000   
17          Kensington Market, Chinatown, Grange Park     0.000000   
18                                      Lawrence Park     0.000000   
19                           Little Portugal, Trinity     0.022222   
20                        Moore Park, Summerhill East     0.000000   
21                  North Toronto West, Lawrence Park     0.055556   
22                             Parkdale, Roncesvalles     0.000000   
23        Queen's Park, Ontario Provincial Government     0.030303   
24                          Regent Park, Harbourfront     0.022727   
25                           Richmond, Adelaide, King     0.000000   
26                                           Rosedale     0.000000   
27                                           Roselawn     0.000000   
28                                 Runnymede, Swansea     0.030303   
29                                     St. James Town     0.000000   
30                        St. James Town, Cabbagetown     0.000000   
31                                     Stn A PO Boxes     0.010417   
32                                    Studio District     0.027027   
33  Summerhill West, Rathnelly, South Hill, Forest...     0.000000   
34                The Annex, North Midtown, Yorkville     0.000000   
35                                        The Beaches     0.000000   
36                       The Danforth West, Riverdale     0.023256   
37           Toronto Dominion Centre, Design Exchange     0.000000   
38                     University of Toronto, Harbord     0.029412   

    Afghan Restaurant  Airport  Airport Food Court  Airport Gate  \
0            0.000000   0.0000              0.0000        0.0000   
1            0.000000   0.0000              0.0000        0.0000   
2            0.000000   0.0000              0.0000        0.0000   
3            0.000000   0.0625              0.0625        0.0625   
4            0.000000   0.0000              0.0000        0.0000   
5            0.000000   0.0000              0.0000        0.0000   
6            0.013333   0.0000              0.0000        0.0000   
7            0.000000   0.0000              0.0000        0.0000   
8            0.000000   0.0000              0.0000        0.0000   
9            0.000000   0.0000              0.0000        0.0000   
10           0.000000   0.0000              0.0000        0.0000   
11           0.000000   0.0000              0.0000        0.0000   
12           0.000000   0.0000              0.0000        0.0000   
13           0.000000   0.0000              0.0000        0.0000   
14           0.000000   0.0000              0.0000        0.0000   
15           0.000000   0.0000              0.0000        0.0000   
16           0.000000   0.0000              0.0000        0.0000   
17           0.0

#### Function that sorts the venues in descending order

In [169]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

#### Top 10 venues for each neighborhood

In [170]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

Neighborhood 1st Most Common Venue  \
0                                        Berczy Park           Coffee Shop   
1       Brockton, Parkdale Village, Exhibition Place                  Café   
2  Business reply mail Processing Centre, South C...         Auto Workshop   
3  CN Tower, King and Spadina, Railway Lands, Har...        Airport Lounge   
4                                 Central Bay Street           Coffee Shop   

  2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
0          Cocktail Bar                Bakery    Seafood Restaurant   
1           Coffee Shop        Breakfast Spot             Nightclub   
2                  Park  Fast Food Restaurant    Light Rail Station   
3       Airport Service           Coffee Shop       Harbor / Marina   
4                  Café        Sandwich Place    Italian Restaurant   

   5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
0         Farmers Market           Cheese Shop              Beer Bar   
1  Performing Arts Venue     Convenience Store                   Bar   
2         Farmers Market      Recording Studio         Burrito Place   
3                  Plane   Rental Car Location      Sculpture Garden   
4            Salad Place      Department Store       Thai Restaurant   

  8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  
0            Restaurant      Greek Restaurant                    Pub  
1                Bakery    Italian Restaurant                Stadium  
2           Pizza Place            Restaurant                Butcher  
3              Boutique                   Bar          Boat or Ferry  
4   Japanese Restaurant          Burger Joint        Bubble Tea Shop

#### Neighborhood clustering - 5 clusters

In [171]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [172]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df_toronto
toronto_merged = toronto_merged.rename(columns={"Neighbourhood":"Neighborhood"})

# merge toronto_grouped with df_toronto to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() 

Postal Code           Borough                                 Neighborhood  \
0         M5A  Downtown Toronto                    Regent Park, Harbourfront   
1         M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government   
2         M5B  Downtown Toronto                     Garden District, Ryerson   
3         M5C  Downtown Toronto                               St. James Town   
4         M4E      East Toronto                                  The Beaches   

    Latitude  Longitude  Cluster Labels 1st Most Common Venue  \
0  43.654260 -79.360636               0           Coffee Shop   
1  43.662301 -79.389494               0           Coffee Shop   
2  43.657162 -79.378937               0        Clothing Store   
3  43.651494 -79.375418               0           Coffee Shop   
4  43.676357 -79.293031               0                   Pub   

  2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
0                   Pub                  Park                Bakery   
1           Yoga Studio                  Café         Smoothie Shop   
2           Coffee Shop                  Café   Japanese Restaurant   
3                  Café          Cocktail Bar            Restaurant   
4                 Trail     Health Food Store          Dance Studio   

         5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
0                      Theater                  Café        Breakfast Spot   
1                     Beer Bar        Sandwich Place            Restaurant   
2               Cosmetics Shop       Bubble Tea Shop                 Hotel   
3                     Beer Bar   American Restaurant             Gastropub   
4  Eastern European Restaurant   Dumpling Restaurant            Donut Shop   

  8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  
0               Brewery                   Spa         Farmers Market  
1   Distribution Center                 Diner  Portuguese Restaurant  
2        Lingerie Store      Ramen Restaurant     Italian Restaurant  
3        Clothing Store    Seafood Restaurant         Cosmetics Shop  
4      Doner Restaurant               Dog Run    Distribution Center

#### Get Toronto coordinates

In [178]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Toronto are 43.6534817, -79.3839347.


#### Visualize clusters in map

In [179]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### Cluster 1 - Coffee shops and food places

In [180]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[2] + list(range(6, toronto_merged.shape[1]))]]

Neighborhood 1st Most Common Venue  \
0                           Regent Park, Harbourfront           Coffee Shop   
1         Queen's Park, Ontario Provincial Government           Coffee Shop   
2                            Garden District, Ryerson        Clothing Store   
3                                      St. James Town           Coffee Shop   
4                                         The Beaches                   Pub   
5                                         Berczy Park           Coffee Shop   
6                                  Central Bay Street           Coffee Shop   
7                                            Christie         Grocery Store   
8                            Richmond, Adelaide, King           Coffee Shop   
9                        Dufferin, Dovercourt Village              Pharmacy   
10  Harbourfront East, Union Station, Toronto Islands           Coffee Shop   
11                           Little Portugal, Trinity                   Bar   
12                       The Danforth West, Riverdale      Greek Restaurant   
13           Toronto Dominion Centre, Design Exchange           Coffee Shop   
14       Brockton, Parkdale Village, Exhibition Place                  Café   
15                     India Bazaar, The Beaches West                  Park   
16                     Commerce Court, Victoria Hotel           Coffee Shop   
17                                    Studio District           Coffee Shop   
20                                   Davisville North          Dance Studio   
22                      High Park, The Junction South    Mexican Restaurant   
23                  North Toronto West, Lawrence Park           Coffee Shop   
24                The Annex, North Midtown, Yorkville        Sandwich Place   
25                             Parkdale, Roncesvalles             Gift Shop   
26                                         Davisville           Pizza Place   
27                     University of Toronto, Harbord                  Café   
28                                 Runnymede, Swansea           Coffee Shop   
30          Kensington Market, Chinatown, Grange Park           Coffee Shop   
31  Summerhill West, Rathnelly, South Hill, Forest...           Coffee Shop   
32  CN Tower, King and Spadina, Railway Lands, Har...        Airport Lounge   
34                                     Stn A PO Boxes           Coffee Shop   
35                        St. James Town, Cabbagetown           Coffee Shop   
36             First Canadian Place, Underground city           Coffee Shop   
37                               Church and Wellesley           Coffee Shop   
38  Business reply mail Processing Centre, South C...         Auto Workshop   

   2nd Most Common Venue          3rd Most Common Venue  \
0                    Pub                           Park   
1            Yoga Studio                           Café   
2            Coffee Shop                           Café   
3                   Café                   Cocktail Bar   
4                  Trail              Health Food Store   
5           Cocktail Bar                         Bakery   
6                   Café                 Sandwich Place   
7                   Café                           Park   
8                   Café                            Gym   
9                 Bakery                           Park   
10              Aquarium                          Hotel   
11           Coffee Shop                    Men's Store   
12           Coffee Shop             Italian Restaurant   
13                 Hotel                     Restaurant   
14           Coffee Shop                 Breakfast Spot   
15            Steakhouse                 Ice Cream Shop   
16            Restaurant                           Café   
17   American Restaurant                         Bakery   
20                 Hotel                           Park   
22       Thai Restaurant                           Café   
23        Clothing Store            

#### Cluster 2 - Trails and playgrounds

In [181]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[2] + list(range(6, toronto_merged.shape[1]))]]

Neighborhood 1st Most Common Venue 2nd Most Common Venue  \
29  Moore Park, Summerhill East                 Trail            Playground   

   3rd Most Common Venue 4th Most Common Venue        5th Most Common Venue  \
29              Wine Bar          Dance Studio  Eastern European Restaurant   

   6th Most Common Venue 7th Most Common Venue 8th Most Common Venue  \
29   Dumpling Restaurant            Donut Shop      Doner Restaurant   

   9th Most Common Venue 10th Most Common Venue  
29               Dog Run    Distribution Center

#### Cluster 3 - Park and Bus Line

In [182]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[2] + list(range(6, toronto_merged.shape[1]))]]

Neighborhood 1st Most Common Venue 2nd Most Common Venue  \
18  Lawrence Park                  Park              Bus Line   

   3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
18           Swim School              Wine Bar         Deli / Bodega   

          6th Most Common Venue 7th Most Common Venue 8th Most Common Venue  \
18  Eastern European Restaurant   Dumpling Restaurant            Donut Shop   

   9th Most Common Venue 10th Most Common Venue  
18      Doner Restaurant                Dog Run

#### Cluster 4 - Parks and trails

In [183]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[2] + list(range(6, toronto_merged.shape[1]))]]

Neighborhood 1st Most Common Venue  \
21  Forest Hill North & West, Forest Hill Road Park      Sushi Restaurant   
33                                         Rosedale                  Park   

   2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
21                  Park         Jewelry Store                 Trail   
33                 Trail            Playground              Wine Bar   

   5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
21              Wine Bar   Distribution Center          Dessert Shop   
33          Dance Studio   Dumpling Restaurant            Donut Shop   

   8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  
21    Dim Sum Restaurant                 Diner         Discount Store  
33      Doner Restaurant               Dog Run    Distribution Center

#### Cluster 5 - Music venues

In [184]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[2] + list(range(6, toronto_merged.shape[1]))]]

Neighborhood 1st Most Common Venue 2nd Most Common Venue  \
19     Roselawn           Music Venue                Garden   

   3rd Most Common Venue 4th Most Common Venue        5th Most Common Venue  \
19              Wine Bar         Deli / Bodega  Eastern European Restaurant   

   6th Most Common Venue 7th Most Common Venue 8th Most Common Venue  \
19   Dumpling Restaurant            Donut Shop      Doner Restaurant   

   9th Most Common Venue 10th Most Common Venue  
19               Dog Run    Distribution Center